# Tweets download

Previously, you need to create a Twitter developer account [here](https://developer.twitter.com/en).

In [2]:
!pip install tweepy

In [3]:
import time
from datetime import datetime

import tweepy

import pymongo

In [4]:
DATE_FORMAT = '%a %b %d %H:%M:%S +0000 %Y'

In [5]:
# Twitter credentials
API_KEY = ''
API_SECRET_KEY = ''

ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''

In [9]:
# MongoDB connection data
HOST = 'localhost'
PORT = '27017'
DB_NAME = 'trending'

### Connecting to database

In [13]:
db = None
try:
    db_client = pymongo.MongoClient('localhost' + ':' + '27017')
    db = db_client['trending']
    print('Conecction established successfully!')
except pymongo.errors.ConnectionFailure:
    print('Connection cannot be established')

Conecction established successfully!


In [14]:
USERS_TIMELINE_COL = db['users_timeline']
HISTORY_COL = db['history']

### Connecting to Twitter

In [15]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)
#stream = tweepy.Stream(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

### Getting latest timeline for an user

In [16]:
# https://docs.tweepy.org/en/latest/api.html#tweepy.API.user_timeline
tweets = api.user_timeline(screen_name = 'IvanDuque', count = 200)

BadRequest: 400 Bad Request
215 - Bad Authentication data.

In [ ]:
for tweet in tweets:
    tweet_dict = tweet._json
    tweet_dict['created_at'] = datetime.strptime(tweet_dict['created_at'], DATE_FORMAT)
    USERS_TIMELINE_COL.insert_one(tweet_dict)

### Getting history for a topic

In [ ]:
id = None
while True:
    try:
        # https://docs.tweepy.org/en/latest/api.html#tweepy.API.search_tweets
        tweets = api.search_tweets(q = 'elecciones colombia', tweet_mode = 'extended', lang = 'es', max_id = id)
        for tweet in tweets:
            tweet_dict = tweet._json
            if not tweet_dict['full_text'].startswith('RT'):
                tweet_dict['created_at'] = datetime.strptime(tweet_dict['created_at'], DATE_FORMAT)
                HISTORY_COL.insert_one(tweet_dict)
        id = tweet.id
    except tweepy.TooManyRequests:
        print('Too many requests. Sleeping...')
        time.sleep(600)
        print('Getting back to action!')
        pass